In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
#loading the training data
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#loading the test data
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_ids = test_data["PassengerId"]
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
def clean(data):
    data = data.drop(["Ticket" , "Cabin" , "Name" , "PassengerId"] , axis =1) #drop the ticket and cabin since it is not so useful for classification
    cols = ["SibSp" , "Parch" , "Fare" , "Age"]
    for col in cols:
        data[col].fillna(data[col].median() , inplace = True)

    data.Embarked.fillna("U" , inplace = True)
    return data
    
train_data = clean(train_data)
test_data = clean(test_data)

In [5]:
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [6]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
cols = ["Sex", "Embarked"]
for col in cols:
    train_data[col] = le.fit_transform(train_data[col])
    test_data[col]=le.fit_transform(test_data[col])
    print(le.classes_)
    

train_data.head()

['female' 'male']
['C' 'Q' 'S']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 

y= train_data["Survived"]
x= train_data.drop(["Survived"] , axis = 1)
x_train , x_val , y_train , y_val = train_test_split ( x , y , test_size=0.2 , random_state=1)

In [8]:
classifier = LogisticRegression(random_state=0 , max_iter = 1000 ).fit(x_train ,y_train)

In [9]:
predictions = classifier.predict(x_val)
from sklearn.metrics import accuracy_score
accuracy_score(y_val , predictions)

0.7988826815642458

In [10]:
submission_predictions = classifier.predict(test_data)

In [11]:
df = pd.DataFrame({"PassengerId": test_ids.values , 
                   "Survived":submission_predictions
                  })

In [12]:
df.to_csv("submission.csv" , index=False)